# Core Assignment 1 - Efficient Yelp API Calls

- Name: Tyler Schelling

- Date: 12/20/2022

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## Load Credentials and Instantiate YelpAPI Variable

In [2]:
with open('/Users/tyler/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

## Efficient API Call

### Define Search Terms

In [4]:
LOCATION = 'San Luis Obispo, CA'
TERM = 'Sandwiches'

In [5]:
JSON_FILE = "Data/results_in_progress_SLO_sandwiches.json"

### Create JSON File

In [7]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder, exist_ok=True)
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
else:
    print(f'[i] {JSON_FILE} already exists')

[i] Data/results_in_progress_SLO_sandwiches.json not found. Saving empty list to file.


### Determine Results in File

In [8]:
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Determine QTY of Pages Needed

In [9]:
query_results = yelp_api.search_query(location = LOCATION,
                                      term = TERM,
                                      offset = n_results)
query_results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
total_results = query_results['total']
total_results

194

In [13]:
results_per_page = len(query_results['businesses'])
results_per_page

20

In [15]:
n_pages = math.ceil((query_results['total'] - n_results) / results_per_page)
n_pages

10

### Add Results to JSON File

In [ ]:
previous_results.extend(query_results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(previous_results, f)

## TQDM Progress Loop Combining Above Section

In [16]:
for i in tqdm_notebook(range(1, n_pages+1)):
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 API calls. Stopping loop.')
        break
    
    query_results = yelp_api.search_query(location = LOCATION,
                                      term = TERM,
                                      offset = n_results)
    previous_results.extend(query_results['businesses'])
    
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)
        
    time.sleep(.2)

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tgx533AzRRPFVdVP2el2rw,high-street-market-and-deli-san-luis-obispo,High Street Market & Deli,https://s3-media1.fl.yelpcdn.com/bphoto/newdKF...,False,https://www.yelp.com/biz/high-street-market-an...,1403,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 35.2723045349121, 'longitude': -1...",[],$$,"{'address1': '350 High St', 'address2': '', 'a...",+18055414738,(805) 541-4738,415.941339
1,UwFyxqcuzzsnCOfp1O-SDg,mr-pickles-sandwich-shop-slo-san-luis-obispo,Mr Pickle's Sandwich Shop - SLO,https://s3-media4.fl.yelpcdn.com/bphoto/2Oo9_R...,False,https://www.yelp.com/biz/mr-pickles-sandwich-s...,310,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 35.2872332, 'longitude': -120.664...","[pickup, delivery]",$,"{'address1': '1075 Olive St', 'address2': '', ...",+18055459909,(805) 545-9909,2072.335797
2,5KNda1IXjbUf8XPfTzWpdQ,lincoln-market-and-deli-san-luis-obispo,Lincoln Market & Deli,https://s3-media2.fl.yelpcdn.com/bphoto/d8w0wP...,False,https://www.yelp.com/biz/lincoln-market-and-de...,288,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.5,"{'latitude': 35.28488636616675, 'longitude': -...",[delivery],$$,"{'address1': '496 Broad St', 'address2': '', '...",+18055433443,(805) 543-3443,1855.486958
3,lfxbCCMgl_PFRhLF_Cm24g,guss-grocery-san-luis-obispo,Gus's Grocery,https://s3-media4.fl.yelpcdn.com/bphoto/YtB-0G...,False,https://www.yelp.com/biz/guss-grocery-san-luis...,455,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.0,"{'latitude': 35.2766429, 'longitude': -120.657...","[pickup, delivery]",$,"{'address1': '1638 Osos St', 'address2': '', '...",+18055438684,(805) 543-8684,1085.072790
4,Ei67jbqDd9fT3oghWiCK7g,broad-street-giant-grinder-san-luis-obispo,Broad Street Giant Grinder,https://s3-media2.fl.yelpcdn.com/bphoto/8yr12t...,False,https://www.yelp.com/biz/broad-street-giant-gr...,205,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",4.5,"{'latitude': 35.2726374106087, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '1901 Broad St', 'address2': None...",+18055436700,(805) 543-6700,653.578148


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
189,Nv1k9Nxxqp-5AOhyznbrAg,dennys-san-luis-obispo,Denny's,https://s3-media3.fl.yelpcdn.com/bphoto/9CdDoT...,False,https://www.yelp.com/biz/dennys-san-luis-obisp...,137,"[{'alias': 'breakfast_brunch', 'title': 'Break...",2.5,"{'latitude': 35.2642538921323, 'longitude': -1...",[],$,"{'address1': '208 Madonna Rd', 'address2': '',...",+18055433138,(805) 543-3138,1233.171881
190,NAtpK4nE7AvtlX1su9swqw,gardens-of-avila-restaurant-avila-beach,Gardens of Avila Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/8DJVW0...,False,https://www.yelp.com/biz/gardens-of-avila-rest...,306,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 35.18598306812193, 'longitude': -...",[],$$,"{'address1': '1215 Avila Beach Dr', 'address2'...",+18055957365,(805) 595-7365,10191.991071
191,n8lPV_DUvMHAT0zT0jxt2A,dominos-pizza-san-luis-obispo,Domino's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/eRII3G...,False,https://www.yelp.com/biz/dominos-pizza-san-lui...,141,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 35.2944, 'longitude': -120.67055}","[pickup, delivery]",$,"{'address1': '866 E Foothill Blvd', 'address2'...",+18055443636,(805) 544-3636,2939.081243
192,WxrVxu80WB55zHbD9qv0-A,starbucks-san-luis-obispo-6,Starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/N3hj95...,False,https://www.yelp.com/biz/starbucks-san-luis-ob...,58,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 35.293715, 'longitude': -120.670984}",[delivery],$$,"{'address1': '17 Chorro St', 'address2': 'Ste ...",+18055479054,(805) 547-9054,2859.814578
193,I-vU31GDWTSX75LbWJe_BA,the-pairing-knife-san-luis-obispo-2,The Pairing Knife,https://s3-media1.fl.yelpcdn.com/bphoto/RBBGBJ...,False,https://www.yelp.com/biz/the-pairing-knife-san...,19,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",3.0,"{'latitude': 35.2454989, 'longitude': -120.596...",[],$,"{'address1': '', 'address2': '', 'address3': '...",+18055447247,(805) 544-7247,6623.452423
